<a href="https://colab.research.google.com/github/DarraghTate/MSc_Advanced_Machine_Learning/blob/main/Assignment_1_Data_Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
! pip install pandas
! pip install sklearn
! pip install numpy
! pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1Lg8a476HB7bSB2gKLv15eanBkHsPU9SD"})
downloaded.GetContentFile('diamonds.csv') 
import pandas as pd

# Used to set random states, for consistency
random_state = 23
# https://drive.google.com/file/d/1Lg8a476HB7bSB2gKLv15eanBkHsPU9SD/view?usp=sharing
# 1Lg8a476HB7bSB2gKLv15eanBkHsPU9SD
df = pd.read_csv('diamonds.csv')

df = df[["carat","cut","color","clarity","depth","table","price","x","y","z"]]
# .astype is pandas categorisation function.
df[['cut', 'color', 'clarity']] = df[['cut', 'color', 'clarity']].astype('category')
# .cat.codes is pandas function to take the categorical information and replace the string variables with the numerics
df['cut'] = df['cut'].cat.codes
df['color'] = df['color'].cat.codes
df['clarity'] = df['clarity'].cat.codes

# x is the dataframe conaining the independant variables
X_values = df[["carat","cut","color","clarity","depth","table","x","y","z"]]
# y is the dataframe containing the dependant variables
y_values = df['price']

categories = pd.qcut(y_values, 10)

y_categorical = pd.DataFrame(y_values, columns={'price'})

y_categorical.loc[(y_categorical['price']<500), "categories"] = "< 500"
y_categorical.loc[(y_categorical['price']>=500) & (y_categorical['price']<1000), 'categories'] = "500 - 1000"
y_categorical.loc[(y_categorical['price']>=1000) & (y_categorical['price']<1500), 'categories'] = "1000 - 1500"
y_categorical.loc[(y_categorical['price']>=1500) & (y_categorical['price']<2000), 'categories'] = "1500 - 2000"
y_categorical.loc[(y_categorical['price']>=2000) & (y_categorical['price']<2500), 'categories'] = "2000 - 2500"
y_categorical.loc[(y_categorical['price']>=2500) & (y_categorical['price']<3000), 'categories'] = "2500 - 3000"
y_categorical.loc[(y_categorical['price']>=3000) & (y_categorical['price']<3500), 'categories'] = "3000 - 3500"
y_categorical.loc[(y_categorical['price']>=3500), "categories"] = "3500+"

In [25]:
# Linear Regression / Polynomial Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

linear_model = LinearRegression(copy_X = True, fit_intercept = True, n_jobs = None, normalize = False)
linear_model.fit(X_values, y_values)
y_pred= linear_model.predict(X_values)
score = r2_score(y_values, y_pred)
print(f"R squared (No polynomials): {score:.5f}\n")

degree = 4
poly_reg_model = PolynomialFeatures(degree = degree)
X_poly = poly_reg_model.fit_transform(X_values)
polynomial_model = LinearRegression()
polynomial_model.fit(X_poly, y_values)
y_poly_prediction = polynomial_model.predict(X_poly)
poly_score = r2_score(y_values, y_poly_prediction)

print(f'R squared with polynomial degree of {degree}: {poly_score:.5f}')

R squared (No polynomials): 0.88507

R squared with polynomial degree of 4: 0.95211


In [26]:
# K-Means Clustering
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size = .33, random_state = random_state)

best_score = 0
best_num_neighbours = 0
for i in range(1, 21):
  model = KNeighborsRegressor(n_neighbors=i)
  model.fit(X_train, y_train.values.ravel())
  model_score = model.score(X_test, y_test)
  print(f'Model score with {i} neighbours: {model_score:.2f}')
  if (model_score > best_score):
    print(f'New best model found')
    best_score = model_score
    best_num_neighbours = i

print(f'Best Model found: {best_num_neighbours} neighbours which gives an r^2 value of {best_score:.2f}')

Model score with 1 neighbours: 0.92
New best model found
Model score with 2 neighbours: 0.94
New best model found
Model score with 3 neighbours: 0.94
New best model found
Model score with 4 neighbours: 0.94
New best model found
Model score with 5 neighbours: 0.95
New best model found
Model score with 6 neighbours: 0.95
Model score with 7 neighbours: 0.94
Model score with 8 neighbours: 0.94
Model score with 9 neighbours: 0.94
Model score with 10 neighbours: 0.94
Model score with 11 neighbours: 0.94
Model score with 12 neighbours: 0.94
Model score with 13 neighbours: 0.94
Model score with 14 neighbours: 0.94
Model score with 15 neighbours: 0.94
Model score with 16 neighbours: 0.94
Model score with 17 neighbours: 0.94
Model score with 18 neighbours: 0.94
Model score with 19 neighbours: 0.94
Model score with 20 neighbours: 0.94
Best Model found: 5 neighbours which gives an r^2 value of 0.95


In [ ]:
# Neural Network
# Score: 0.943772889411137

import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size = .33)

scaling = StandardScaler().fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

hidden_layer_sizes = (10,20,40,80)
max_iter = 10000
activation = 'relu'
solver = 'adam'
nn_reg = MLPRegressor(max_iter=max_iter, hidden_layer_sizes = hidden_layer_sizes, activation=activation, solver=solver, random_state=random_state)

nn_reg.fit(X_train, y_train.values.ravel())

y_pred = nn_reg.predict(X_test)
print(r2_score(y_test, y_pred))

0.943772889411137


In [29]:
#Support Vector Machine 
#Score: 0.9420355915848466
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.svm import LinearSVC, SVC
from sklearn.svm import SVR

X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size = .33, random_state = random_state)
scaling = StandardScaler().fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)
max_iter = 500000
C = 1000 # 1000 & rbf = 0.9420355915848466
#kernel = 'poly' # 0.8659278159023867
#kernel = 'linear' # 0.8649820116013944
kernel = 'rbf' #0.9073751689826052
degree = 1
svm_model = SVR(max_iter = max_iter, C=C, kernel=kernel, degree = degree)
svm_model.fit(X_train, y_train)
print(svm_model.score(X_test, y_test))




0.9420355915848466


In [ ]:
# Ensemble - Random Forest Regressor
# Score: 0.9803378454248586
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size = .33, random_state = random_state)
scaling = MinMaxScaler(feature_range=(0, 1)).fit(X_train)
ensemble_model = RandomForestRegressor()
ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)
print(ensemble_model.score(X_test, y_test))

0.9803378454248586
